<a href="https://colab.research.google.com/github/JONNY-ME/Microsoft-Rice-Disease-Classification-Challenge/blob/main/lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pytorch-lightning --quiet
!pip install transformers --quiet
!pip install albumentations==0.4.6 --quiet
!pip install timm --quiet

     |████████████████████████████████| 701 kB 39.0 MB/s 
     |████████████████████████████████| 419 kB 73.2 MB/s 
     |████████████████████████████████| 141 kB 66.1 MB/s 
     |████████████████████████████████| 596 kB 70.1 MB/s 
     |████████████████████████████████| 5.9 MB 58.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.
     |████████████████████████████████| 4.7 MB 38.4 MB/s 
     |████████████████████████████████| 101 kB 11.1 MB/s 
     |████████████████████████████████| 6.6 MB 54.8 MB/s 
     |████████████████████████████████| 117 kB 18.6 MB/s 
     |████████████████████████████████| 509 kB 36.6 MB/s 


In [3]:
%%time
!unzip -o -q '/content/drive/MyDrive/Microsoft-rice/Images.zip' -d "/content/Images"

CPU times: user 228 ms, sys: 41.1 ms, total: 269 ms
Wall time: 34.5 s


In [4]:
# import os
# from PIL import Image
# from tqdm.notebook import tqdm

# ipath = '/content/Images/'
# rgn_images = [i for i in os.listdir(ipath) if i.endswith('_rgn.jpg')]
# size = (533, 400)

# for img_path in tqdm(rgn_images):
#     _path = ipath+img_path
#     img = Image.open(_path)
#     img.thumbnail(size, Image.ANTIALIAS)
#     img.save(_path)

In [5]:
import cv2
import os
import torch
import torchvision
import timm
import pandas as pd
import numpy as np
import pytorch_lightning as pl
import albumentations as A
import warnings
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, ReduceLROnPlateau
from pytorch_lightning import LightningDataModule
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import WandbLogger
from torchvision import transforms
from torchmetrics import Accuracy
from transformers import AdamW
from sklearn.model_selection import StratifiedKFold
from albumentations.pytorch import ToTensorV2

warnings.filterwarnings('ignore')

seed = 21
seed_everything(seed)

INFO:pytorch_lightning.utilities.seed:Global seed set to 21


21

In [6]:
path = "/content/drive/MyDrive/Microsoft-rice/"
Train = pd.read_csv(path+'Train.csv')
Test = pd.read_csv(path+'Test.csv')
print(Train.shape, Test.shape)
Train.head(2)

(5340, 2) (2290, 1)


,Image_id,Label
0,id_004wknd7qd.jpg,blast
1,id_004wknd7qd_rgn.jpg,blast


In [7]:
# drop the RGN images
Train = Train[Train.index%2 == 0].reset_index(drop=True)
Test = Test[Test.index%2 == 0].reset_index(drop=True)

# encode the target
label_map = dict(zip(Train.Label.unique(), range(Train.Label.nunique())))
Train.Label = Train.Label.map(label_map)

# add image paths
Train['image_path'] = '/content/Images/' + Train.Image_id
Test['image_path'] = '/content/Images/' + Test.Image_id

Train.head(2)

,Image_id,Label,image_path
0,id_004wknd7qd.jpg,0,/content/Images/id_004wknd7qd.jpg
1,id_005sitfgr2.jpg,1,/content/Images/id_005sitfgr2.jpg


In [9]:
skf = StratifiedKFold(5, shuffle=True, random_state=seed)
X = Train.drop(columns='Label')
y = Train.Label

for fold, (_, valid_index) in enumerate(skf.split(X, y)):
  Train.loc[valid_index, "fold"] = fold

Train.fold = Train.fold.astype(int)
Train.head()

,Image_id,Label,image_path,fold
0,id_004wknd7qd.jpg,0,/content/Images/id_004wknd7qd.jpg,3
1,id_005sitfgr2.jpg,1,/content/Images/id_005sitfgr2.jpg,2
2,id_00stp9t6m6.jpg,0,/content/Images/id_00stp9t6m6.jpg,4
3,id_012zxewnhx.jpg,0,/content/Images/id_012zxewnhx.jpg,1
4,id_0186qwq2at.jpg,2,/content/Images/id_0186qwq2at.jpg,2


In [10]:
# save the new dataframes

train_csv = "/content/Train_mod.csv"
test_csv = "/content/Test_mod.csv"
Train.to_csv(train_csv, index=False) 
Test.to_csv(test_csv, index=False) 

In [12]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def get_train_transforms(pretrained=True):
    augmentations = [
        A.HorizontalFlip(p=.5),
        A.VerticalFlip(p=.5),
        A.ImageCompression(quality_lower=99, quality_upper=100),
        A.ShiftScaleRotate(
            shift_limit=0.2, scale_limit=0.2, 
            rotate_limit=45, border_mode=0, p=.5
        ),
        A.Resize(IMG_HEIGHT, IMG_WIDTH),
        A.Cutout(
            max_h_size=int(IMG_HEIGHT*0.4),
            max_w_size=int(IMG_WIDTH*0.4),
            num_holes=1,
            p=.5,
        ),
    ]
    if pretrained:
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))
    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)


def get_valid_transforms(pretrained=True):
    augmentations = [A.Resize(IMG_HEIGHT, IMG_WIDTH)]
    if pretrained:
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))
    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)



def get_transforms(phase):
    if phase == "train":
        return get_train_transforms()
        
    elif phase in ["valid", "test"]:
        return get_valid_transforms()


In [13]:
class MyDataset(Dataset):

    def __init__(self, df, phase="train", transforms=None, save_data=True, both_images=True):
        self.df = df
        self.image_paths = df['image_path'].values
        if phase in ['train', 'valid']:
            self.targets = df['Label'].values
            if save_data:
                self.images = [
                    self.load_image(image_path)
                    for image_path in tqdm(self.image_paths)
                ]
        self.phase = phase
        self.transforms = transforms
        self.save_data = save_data
        self.both_images = both_images
        
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        if self.save_data and self.phase in ['train', 'valid']:
            if self.both_images:
                image, image2 = self.images[index]
            else:
                image = self.images[index]

        else:
            if self.both_images:
                image, image2 = self.load_image(self.image_paths[index])
            else:
                image = self.load_image(self.image_paths[index])

        if self.transforms:
            image = self.transforms(image=image)["image"]
            if self.both_images:
                image2 = self.transforms(image=image2)["image"]
        full_image = image
        if self.both_images:
          full_image = tuple([image, image2])

        if self.phase in ["train", "valid"]:
            return full_image, torch.tensor(self.targets[index], dtype=torch.long)
        else:
            return full_image

    def load_image(self, image_path):
        image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        if self.both_images:
            path2 = image_path.replace('.jpg', '_rgn.jpg')
            image2 = cv2.cvtColor(cv2.imread(path2), cv2.COLOR_BGR2RGB)
        
            return tuple([image, image2])

        else:
            return image

def get_dataset(df, phase, apply_transforms=True, both_images=True):
    transforms = get_transforms(phase) if apply_transforms else None
    return MyDataset(df, phase, transforms, save_data=False, both_images=both_images) # save data True for fast access but requires RAM

In [14]:
class MyLightningDataModule(LightningDataModule):
    def __init__(self, fold=0, both_images=True):
        super().__init__()
        self.fold = fold
        self.both_images = both_images

    def _split_train_and_valid_df(self):
        df = pd.read_csv(train_csv)

        train_df = df[df["fold"] != self.fold].reset_index(drop=True)
        valid_df = df[df["fold"] == self.fold].reset_index(drop=True)

        return train_df, valid_df

    def setup(self, stage):
        self.train_df, self.valid_df = self._split_train_and_valid_df()

    def _get_dataframe(self, phase):
        if phase == "train":
            return self.train_df
        elif phase == "valid":
            return self.valid_df
        elif phase == "test":
            test_df = pd.read_csv(test_csv)
            return test_df

    def _get_dataset(self, phase):
        df = self._get_dataframe(phase)
        return get_dataset(df, phase, both_images=self.both_images)

    def _get_dataloader(self, phase):
        dataset = self._get_dataset(phase=phase)
        return DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=phase == "train",
            drop_last=phase == "train",
            pin_memory=True,
        )

    def len_dataloader(self, phase):  
        return len(self._get_dataloader(phase=phase))

    def train_dataloader(self):
        return self._get_dataloader(phase="train")

    def val_dataloader(self):
        return self._get_dataloader(phase="valid")

    def test_dataloader(self):
        return self._get_dataloader(phase="test")

    def predict_dataloader(self):
        return self._get_dataloader(phase="test")
    
    def teardown(self, stage):
        # clean up after fit or test
        # called on every process in DDP
        pass

In [15]:
class Head(nn.Module):
    def __init__(self, in_features, out_features):
        super(Head, self).__init__()
        self.head = nn.Linear(in_features=in_features, out_features=out_features)

    def forward(self, x):
        return self.head(x)


class Net(nn.Module):
    def __init__(
        self,
        base_model='convnext_base_384_in22ft1k',
        pretrained=True,
        checkpoint_path=None,
        num_classes=32,
    ):
        super(Net, self).__init__()

        self.backbone = timm.create_model(
            base_model, pretrained=pretrained, checkpoint_path=checkpoint_path
        )
        in_features = self.backbone.get_classifier().in_features
        self.backbone.reset_classifier(num_classes=0, global_pool="avg")
        self.head = Head(in_features=in_features, out_features=num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        return x



class MyModel(LightningModule):
    def __init__(self, learning_rate=2e-4):
        super().__init__()
        
        # log hyperparameters
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.modelx = Net(num_classes=3)
        self.modelr = Net(num_classes=64)
        self.out = nn.Linear(192, 3)
        self.drop = nn.Dropout(0.5)
        self.accuracy = Accuracy()

    # will be used during inference
    def forward(self, x):
      #  x, x_rgn = x
       x = self.modelx(x)
      #  x_rgn = self.modelr(x_rgn)
      #  x = torch.cat((x, x_rgn), 1)
      #  x = self.drop(x)
      #  return F.log_softmax(self.out(x))
       return F.log_softmax(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        
        # training metrics
        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)

        # validation metrics
        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss
    
    def predict_step(self, batch, batch_idx):
        x = batch
        logits = self(x)

        return logits
    
    def configure_optimizers(self):
        optimizer = AdamW(
            self.parameters(), 
            lr=self.learning_rate, 
            betas=[0.9, 0.999],
            eps=1e-8, 
            weight_decay=1e-8
          )
        reduce_lr = ReduceLROnPlateau(
            optimizer=optimizer,
            mode='min',
            factor=0.75,
            patience=2,
            threshold=1e-4,
            threshold_mode='rel',
            cooldown=0,
            min_lr=1e-7,
            eps=1e-8
          )
        scheduler = CosineAnnealingWarmRestarts(
            optimizer=optimizer,
            T_0=10,
            T_mult=1,
            eta_min=1e-6,
            last_epoch=-1
          )
        scheduler_dict = {
            "scheduler": scheduler,
            "interval": 'epoch',
            "monitor": 'val_loss',
        }
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler_dict,
        }


In [20]:
import gc
import inspect
import shutil

def remove_dir(path):
  try:
    shutil.rmtree(path)
  except:
    pass


def free_memory(to_delete: list):
    calling_namespace = inspect.currentframe().f_back

    for _var in to_delete:
        calling_namespace.f_locals.pop(_var, None)
        gc.collect()
        torch.cuda.empty_cache()

def get_callbacks(fold):
  early_stop_callback = pl.callbacks.EarlyStopping(
          monitor="val_loss",
          patience=10,
          verbose=False,
          mode='min',
          strict=True,
          check_finite=True,
          check_on_train_epoch_end=False
  )
  checkpoint_callback = pl.callbacks.ModelCheckpoint(
          filename=f"{fold}",
          monitor='val_loss',
          verbose=True,
          save_last=False,
          save_top_k=1,
          mode='min',
          save_weights_only=True
  )
  lr_monitor_callback = pl.callbacks.LearningRateMonitor(
          log_momentum=False
  )

  fine_tuning_callback = pl.callbacks.BackboneFinetuning(
    unfreeze_backbone_at_epoch=15,
    backbone_initial_ratio_lr=.1
    )
  return [
      early_stop_callback,
      checkpoint_callback,
      lr_monitor_callback,
      # fine_tuning_callback
  ]


In [21]:
remove_dir('/content/lightning_logs')
predictions = []

for fold in [0, 1, 2, 3, 4]:
  print(f"FOLD {fold}")
  module = MyLightningDataModule(fold=fold, both_images=False)
  module.setup(None)


  model = MyModel(learning_rate=2e-4)

  # Initialize Callbacks
  callbacks = get_callbacks(fold)
  
  # Initialize a trainer
  trainer = Trainer(
      accumulate_grad_batches=1,
      amp_backend='native',
      benchmark=False,
      callbacks=callbacks,
      deterministic=True,
      fast_dev_run=False,
      gpus=1,
      gradient_clip_val=0,
      gradient_clip_algorithm='norm',
      # logger=wandb_logger,
      max_epochs=12,
      num_sanity_val_steps=0,
      precision=16,
      # stochastic_weight_avg=False,
  )

  # Train the model ⚡🚅⚡
  trainer.fit(model, module)

  # model inference on the test set ⚡⚡
  model.eval()
  pred = trainer.predict(dataloaders=module.test_dataloader())
  predictions.append(np.exp(np.concatenate(pred)))

  free_memory([trainer, module, model])


FOLD 0


Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_1k_384.pth" to /root/.cache/torch/hub/checkpoints/convnext_base_22k_1k_384.pth
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type     | Params
--------------------------------------
0 | modelx   | Net      | 87.6 M
1 | modelr   | Net      | 87.6 M
2 | out      | Linear   | 387   
3 | drop     | Dropout  | 0     
4 | accuracy | Accuracy | 0     
--------------------------------------
175 M     Trainable para

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 66: 'val_loss' reached 0.48816 (best 0.48816), saving model to '/content/lightning_logs/version_0/checkpoints/0.ckpt' as top 1


...

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 10, global step 726: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 11, global step 792: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=12` reached.
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_4/checkpoints/4.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/lightning_logs/version_4/checkpoints/4.ckpt


Predicting: 66it [00:00, ?it/s]

In [ ]:
# from tqdm.notebook import tqdm
# import numpy as np

# def load_model(exp_dir, fold):
#     ckpt_path = os.path.join(
#         exp_dir, "checkpoints", f"{fold}.ckpt"
#     )
#     model = MyModel.load_from_checkpoint(ckpt_path)

#     return model

# def inference(model, dataloader):
#     inferences = []
#     with torch.inference_mode():
#         for x in tqdm(dataloader):
#             logits = model(x.to("cuda")).cpu().numpy()
#             inferences.append(logits)

#     return np.exp(np.concatenate(inferences))

# drs = [
#     (f'/content/lightning_logs/version_{i}', j) 
#     for i, j in [(5, 3)]
# ]


# pps = []
# for dr, fold in drs:
#   model = load_model(dr, fold)
#   model.to('cuda')
#   module = MyLightningDataModule(fold=fold, both_images=False)
#   module.setup(None)
#   pps.append(inference(model, dataloader=module.test_dataloader()))

  0%|          | 0/36 [00:00<?, ?it/s]

In [25]:
pred = np.mean(predictions, axis=0)

sub = Test[['Image_id']]
sub[list(label_map.keys())] = pred


sub.to_csv('/content/lightning.csv', index=False)
sub

,Image_id,blast,brown,healthy
0,id_00vl5wvxq3.jpg,0.999905,0.000020,0.000018
1,id_01hu05mtch.jpg,0.002174,0.997646,0.000042
2,id_030ln10ewn.jpg,0.495295,0.494037,0.008724
3,id_03z57m8xht.jpg,0.999863,0.000012,0.000006
4,id_04ngep1w4b.jpg,0.999303,0.000206,0.000349
...,...,...,...,...
1140,id_zrdlgjrq3r.jpg,0.000194,0.003647,0.995865
1141,id_zsfayxwipp.jpg,0.000049,0.998854,0.000815
1142,id_ztvp2l9k3h.jpg,0.997003,0.002466,0.000018
1143,id_zwwcma7hlt.jpg,0.999809,0.000018,0.000106
